In [4]:
!nvidia-smi

Fri Nov 26 09:31:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 455.38       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3080    Off  | 00000000:01:00.0  On |                  N/A |
| 53%   31C    P8    36W / 340W |    281MiB / 10018MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Work in Progress

This is an implementation of RGCNN from the paper https://github.com/tegusi/RGCNN

Initial testing will be done on the GeometricShapes dataset from PyG

In [55]:
import torch
import torch_geometric
from torch_geometric.datasets import GeometricShapes

dataset = GeometricShapes(root='data/GeometricShapes')
print("The whole dataset:       \t", dataset)
print("Example from the dataset:\t", dataset[0])

The whole dataset:       	 GeometricShapes(40)
Example from the dataset:	 Data(pos=[32, 3], face=[3, 30], y=[1])


Converting the Meshes from the dataset to a PointCloud:

In [56]:
from torch_geometric.transforms import SamplePoints

# For reproductibility
torch.manual_seed(42)
points_number = 1024 
dataset.transform =  SamplePoints(num=points_number)
print("Example of Point Cloud: \t", dataset[0])

Example of Point Cloud: 	 Data(pos=[1024, 3], y=[1])


The following classes are defined as in the RGCNN github:

In [57]:
import torch.nn as nn

class GetGraph(nn.Module):
    def __init__(self):
        super(GetGraph, self).__init__()

    def forward(self, point_cloud):
        point_cloud_transpose = point_cloud.permute(0, 2, 1)
        point_cloud_inner = torch.matmul(point_cloud, point_cloud_transpose)
        point_cloud_inner = -2 * point_cloud_inner
        point_cloud_square = torch.sum(torch.mul(point_cloud, point_cloud), dim=2, keepdim=True)
        point_cloud_square_tranpose = point_cloud_square.permute(0, 2, 1)
        adj_matrix = point_cloud_square + point_cloud_inner + point_cloud_square_tranpose
        adj_matrix = torch.exp(-adj_matrix)
        return adj_matrix


class GetLaplacian(nn.Module):
    def __init__(self, normalize=True):
        super(GetLaplacian, self).__init__()
        self.normalize = normalize

    def diag(self, mat):
        # input is batch x vertices
        d = []
        for vec in mat:
            d.append(torch.diag(vec))
        return torch.stack(d)

    def forward(self, adj_matrix):
        if self.normalize:
            D = torch.sum(adj_matrix, dim=2)
            eye = torch.ones_like(D)
            eye = self.diag(eye)
            D = 1 / torch.sqrt(D)
            D = self.diag(D)
            L = eye - torch.matmul(torch.matmul(D, adj_matrix), D)
        else:
            D = torch.sum(adj_matrix, dim=1)
            D = torch.matrix_diag(D)
            L = D - adj_matrix
        return L

Testing the functions:

In [58]:
get_graph = GetGraph()
print("Shape of the dataset points: \t", dataset[0].pos.shape)
data=dataset[0].pos
data=data.reshape([1,data.shape[0], data.shape[1]])
print("New shape:                   \t", data.shape)
x=data
W = get_graph(data)

W_torch = W
print("Graph shape:                 \t", W_torch.shape)
W=torch.squeeze(W)
print("Reduction of the data dimension:\t", W.shape)
print("Graph:                       \t", W)


getter_laplacian = GetLaplacian()
L_torch = getter_laplacian(W_torch)
L = torch.squeeze(L_torch)
print("Laplacian shape:             \t", L_torch.shape)
print("Squeezed Laplacian shape     \t", L.shape)
print("Laplacian                    \t", L)


Shape of the dataset points: 	 torch.Size([1024, 3])
New shape:                   	 torch.Size([1, 1024, 3])
Graph shape:                 	 torch.Size([1, 1024, 1024])
Reduction of the data dimension:	 torch.Size([1024, 1024])
Graph:                       	 tensor([[1.0000, 0.5828, 0.9376,  ..., 0.1363, 0.3722, 0.7379],
        [0.5828, 1.0000, 0.6627,  ..., 0.5064, 0.2619, 0.6291],
        [0.9376, 0.6627, 1.0000,  ..., 0.2344, 0.5538, 0.9149],
        ...,
        [0.1363, 0.5064, 0.2344,  ..., 1.0000, 0.2453, 0.3585],
        [0.3722, 0.2619, 0.5538,  ..., 0.2453, 1.0000, 0.7663],
        [0.7379, 0.6291, 0.9149,  ..., 0.3585, 0.7663, 1.0000]])
Laplacian shape:             	 torch.Size([1, 1024, 1024])
Squeezed Laplacian shape     	 torch.Size([1024, 1024])
Laplacian                    	 tensor([[ 9.9728e-01, -1.5304e-03, -2.2361e-03,  ..., -3.4304e-04,
         -7.8977e-04, -1.5904e-03],
        [-1.5304e-03,  9.9746e-01, -1.5274e-03,  ..., -1.2321e-03,
         -5.3711e-04, -1.310

Defining the Chebyshev approximation:

In [59]:
import torch.nn as nn
def find_eigmax(L):
    with torch.no_grad():
        e1, _ = torch.eig(L, eigenvectors=False)
        return torch.max(e1[:, 0]).item()

def chebyshev_Lapl(X, Lapl, thetas, order):
    list_powers = []
    nodes = Lapl.shape[0]

    T0 = X.float()

    eigmax = find_eigmax(Lapl)
    L_rescaled = (2 * Lapl / eigmax) - torch.eye(1024)

    y = T0 * thetas[0]
    list_powers.append(y)
    T1 = torch.matmul(L_rescaled, T0)
    list_powers.append(T1 * thetas[1])

    # Computation of: T_k = 2*L_rescaled*T_k-1  -  T_k-2
    for k in range(2, order):
        T2 = 2 * torch.matmul(L_rescaled, T1) - T0
        list_powers.append((T2 * thetas[k]))
        T0, T1 = T1, T2
    y_out = torch.stack(list_powers, dim=-1)
    # the powers may be summed or concatenated. i use concatenation here
    y_out = y_out.view(nodes, -1) # -1 = order* features_of_signal
    return y_out

Testing the Chebyshev approx:

In [60]:
features_1 = 3          # input features number for the 'model'
out_features_1 = 128    # output features number from the first convolutional layer
K1 = 6                  # order of the Chebyshev Polynomial
thetas_1 = nn.Parameter(torch.rand(points_number))  # initialization of the parameters of the model for the first conv layer
out = chebyshev_Lapl(x,L,thetas_1,K1)           # computing the approximate polynomial
print('cheb approx out powers concatenated:', out.shape)    
linear_1 = nn.Linear(K1*features_1, out_features_1)     # MLP == Linear
layer_out_1 = linear_1(out)
print('Layers output:', layer_out_1.shape)

cheb approx out powers concatenated: torch.Size([1024, 18])
Layers output: torch.Size([1024, 128])


Here I will try to define a convolutional layer:

In [61]:
import torch_geometric.nn.conv as cnv
import torch_geometric.utils as tgu

class RGCNN_Conv(nn.Module):
    def __init__(self, F_in, K, F_out):
        super(RGCNN_Conv, self).__init__()
        self.F_in = F_in
        self.K = K
        self.F_out = F_out

        self.W = nn.Parameter(torch.Tensor(self.K * self.F_in, self.F_out))
        nn.init.normal_(self.W, mean=0, std=0.2)

        self.B = nn.Parameter(torch.Tensor(self.F_out))
        nn.init.normal_(self.B, mean=0, std=0.2)

        self.cheb_conv = cnv.ChebConv(in_channels=F_in, out_channels=F_out, K=K, normalization="sym", bias=True)

        self.relu = nn.ReLU()

    def forward(self, x, W):
        N, M, F_in = list(x.size())
        edge_index, edge_weight = tgu.dense_to_sparse(W)
        x = self.cheb_conv(x=x, edge_index=edge_index, edge_weight=edge_weight)
        x = self.relu(x)
        return x.reshape(N, M, self.F_out)


TESTS:

In [48]:
x = dataset[0].pos
x = x.reshape(1, x.shape[0], x.shape[1])
get_graph = GetGraph()
W = get_graph(x)
edge_index, edge_weight = tgu.dense_to_sparse(W)
conv1 = RGCNN_Conv(F_in=3, K=6, F_out=128)
aux = conv1(x, W)
#print(aux.shape)

print(conv1.state_dict())   # This shows the Parameters of the Module

OrderedDict([('W', tensor([[-0.4086, -0.1569, -0.1427,  ..., -0.0811, -0.0623,  0.0787],
        [ 0.0311, -0.1582, -0.0621,  ..., -0.0046,  0.0769, -0.1732],
        [-0.3296,  0.2379, -0.1198,  ...,  0.3813,  0.0613, -0.1485],
        ...,
        [-0.0922, -0.2568,  0.2475,  ...,  0.1912, -0.3486, -0.1763],
        [ 0.0580,  0.2378, -0.0808,  ..., -0.1897,  0.3070,  0.0199],
        [ 0.0563, -0.0326,  0.3251,  ...,  0.2126, -0.2054,  0.4179]])), ('B', tensor([-0.5937, -0.0998,  0.1442, -0.3224, -0.1982,  0.1147,  0.0350,  0.3240,
        -0.3171, -0.1003,  0.0058,  0.1605,  0.0485,  0.5452, -0.1594, -0.0728,
        -0.0137, -0.2377,  0.1565,  0.1457,  0.3290,  0.0669,  0.1576, -0.3050,
         0.5961,  0.3220, -0.1121, -0.0196,  0.1485, -0.4267, -0.0749, -0.2158,
         0.1623, -0.0470, -0.0533,  0.2456, -0.1200, -0.1967, -0.0663, -0.1569,
         0.2635, -0.3162,  0.2360, -0.2630,  0.0182, -0.0955, -0.1706, -0.2998,
         0.2625,  0.2627,  0.0269,  0.1118, -0.2047,  0.024

With the defined Convolutional Layer we can create our model:

In [62]:
class RGCNN_Model(nn.Module):
    def __init__(self, points_number, F, K, M, in_features):

        # Verify the consistency w.r.t. the number of layers.
        assert len(F) == len(K)
        
        super(RGCNN_Model, self).__init__()
        self.points_number = points_number
        self.F = F
        self.L = L
        self.M = M
        self.layers_number = len(F)
        self.get_graph = GetGraph()
        self.convolution_list = []
        self.mlp_list = []
        self.rloss = nn.MSELoss()
        self.pool = nn.MaxPool1d(points_number)
        self.relu = nn.ReLU()

        for i in range(self.layers_number):
            if i == 0:
                layer = RGCNN_Conv(in_features, K[i], F[i])
            else:
                layer = RGCNN_Conv(F[i-1], K[i], F[i])
            setattr(self, 'gcn%d' % i, layer)

        for i in range(len(M)):
            if i == 0:
                mlp = nn.Linear(F[-1], M[i])
            else:
                mlp = nn.Linear(M[i-1], M[i])
            setattr(self, 'mlp%d' % i, mlp )

    def forward(self, x, label):
        W = self.get_graph(x)
        losses = []
        labels = torch.zeros(40)
        labels[label] = 1
        labels = labels.to(device)
        for i in range(self.layers_number):
            x = getattr(self, 'gcn%d' % i)(x, W)
        loss = self.rloss(x, labels)
        x = x.permute(0, 2, 1)
        x = self.pool(x)
        x.squeeze_(2)
        for i in range(len(self.M)):
            x = getattr(self, 'mlp%d' % i)(x)
            x = self.relu(x)
        return x, loss

In [63]:
F = [128, 512, 1024, 512, 128, 40]  # Number of graph convolutional filters.
K = [6, 5, 3, 1, 1, 1]  # Polynomial orders.
M = [512, 128, 40]  # Output dimensionality of fully connected layers.
criterion = torch.nn.CrossEntropyLoss()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#device = 'cpu'
print("Training on ", device)

model = RGCNN_Model(points_number=points_number, F=F, K=K, M=M, in_features=3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)

Training on  cuda


In [64]:
def train(train_loader):
    model.train()
    
    for data in train_loader:
        optimizer.zero_grad()
        X = data.pos
        label = data.y
        X = X.reshape([1, X.shape[0], X.shape[1]])
        X = X.to(device)
        label = label.to(torch.float)
        label = label.to(device)
        # Forward pass.
        print(label.to(torch.int).item())
        out, loss = model(X, label.to(torch.int).item())
        # Calculate gradients.
        loss.backward()
        # Updates the models parameters
        optimizer.step()

print(dataset[0].pos.shape)
train(dataset)

torch.Size([1024, 3])
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


In [65]:
def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        X = data.pos
        labels = data.y
        X, labels = X.to(device), labels.to(device)  
        X = X.reshape(1, X.shape[0], X.shape[1])
        # Forward pass.
        out, loss = model(X, labels)
        # Take the index of the class with the highest probability.
        pred = out.argmax(dim=1)
        # Compare with ground-truth labels.
        correct += int((pred == labels).sum()) 
        print(correct)
        # print(len(loader))
    return correct / len(loader)

test(dataset)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


0.0

In [67]:
for epoch in range(1, 101):
    train(dataset)
    train_acc = test(dataset)
    print(f'Epoch: {epoch}, Train Acc: {train_acc:.4f}')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
Epoch: 1, Train Acc: 0.0250
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
Epoch: 2, Train Acc: 0.0000
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
Epoch: 3, Train Acc: 0.0250
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


KeyboardInterrupt: 